# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/irisruz/notebook9bf431d1ed/edit)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn.functional as F
import torch.cuda

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-books-reviews/books_data.csv
/kaggle/input/amazon-books-reviews/Books_rating.csv


In [2]:
dataset_path = '/kaggle/input/amazon-books-reviews/Books_rating.csv'
df = pd.read_csv(dataset_path).sample(n=30000)
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
1361968,0192817728,The Secret Garden (Worlds Classics),NaN,A2IUOLYIYWI77S,"K. C. Skaggs ""K farrell""",0/0,5.0,1295568000,Great again decades later.,I was excited to find this book accessible and...
612376,B0006ASRNG,Christ and culture,NaN,A3TIJC6L8USJ6Q,Elderbear,50/56,5.0,984528000,The Sacred and the Profane: 5 Ways of Relating,"H. Richard Niebuhr writes as a Christian, but ..."
2415963,B000OUU29A,Raging Bull,NaN,AN8E27TFTKGBC,Chris Pascale,0/0,5.0,1314057600,Perhaps the Most Honest Autobiography Ever Wri...,My title may be entrenched in the idea that wh...
2093063,1582790337,Wuthering Heights,NaN,A3JC7J57DZHOVK,Jan,1/1,5.0,1354579200,A Classic: Wuthering Heights,I am trying to read the classics now. I rememb...
2958726,B000FML2C8,"Lord of the Flies, a Novel",NaN,A1I1WPZYT7W405,Karin Verbeek,2/2,5.0,1200873600,Even better the second time around,"I read Lord of the Flies in my english class, ..."


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
columns_to_keep = ['Title','review/score','review/text']
df = df[columns_to_keep]
df.head()

,Title,review/score,review/text
1361968,The Secret Garden (Worlds Classics),5.0,I was excited to find this book accessible and...
612376,Christ and culture,5.0,"H. Richard Niebuhr writes as a Christian, but ..."
2415963,Raging Bull,5.0,My title may be entrenched in the idea that wh...
2093063,Wuthering Heights,5.0,I am trying to read the classics now. I rememb...
2958726,"Lord of the Flies, a Novel",5.0,"I read Lord of the Flies in my english class, ..."


In [5]:
def categorize_rating(rating):
    if rating >= 4:
        return 'positive'
    elif rating <= 2:
        return 'negative'
    else:
        return 'neutral'
df['sentiment'] = df['review/score'].apply(categorize_rating)

df.head()

,Title,review/score,review/text,sentiment
1361968,The Secret Garden (Worlds Classics),5.0,I was excited to find this book accessible and...,positive
612376,Christ and culture,5.0,"H. Richard Niebuhr writes as a Christian, but ...",positive
2415963,Raging Bull,5.0,My title may be entrenched in the idea that wh...,positive
2093063,Wuthering Heights,5.0,I am trying to read the classics now. I rememb...,positive
2958726,"Lord of the Flies, a Novel",5.0,"I read Lord of the Flies in my english class, ...",positive


In [6]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=50

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
classes = df.sentiment.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'positive': 0, 'negative': 1, 'neutral': 2}
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [8]:
df['label'] = df['sentiment'].apply(lambda x: class_tok2idx[x])
df.head()

,Title,review/score,review/text,sentiment,label
1361968,The Secret Garden (Worlds Classics),5.0,I was excited to find this book accessible and...,positive,0
612376,Christ and culture,5.0,"H. Richard Niebuhr writes as a Christian, but ...",positive,0
2415963,Raging Bull,5.0,My title may be entrenched in the idea that wh...,positive,0
2093063,Wuthering Heights,5.0,I am trying to read the classics now. I rememb...,positive,0
2958726,"Lord of the Flies, a Novel",5.0,"I read Lord of the Flies in my english class, ...",positive,0


In [9]:
sequence_0 = "This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paintings by Olivia.If you're looking for heavy literary content, this isn't the place to find it -- there's only about 2 pages with text and everything else is photos.Bottom line: if you only want one book, the Six Foot One ... is probably a better choice, however, if you like Julie like I like Julie, you won't go wrong on this one either."
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq0 is 115
{'input_ids': tensor([[  101,  1188,  1110,  1178,  1111,  7129,  1457, 11098,  3899,   119,
          1135,   112,   188,   170,  2436,  1104,  1123,  7630,   118,   118,
          1164,  2908,  5097,  3869,  1114,   170,  3505,  2237,  1104,  4694,
          1118,  7247,   119,  1409,  1128,   112,  1231,  1702,  1111,  2302,
          4618,  3438,   117,  1142,  2762,   112,   189,  1103,  1282,  1106,
          1525,  1122,   118,   118,  1175,   112,   188,  1178,  1164,   123,
          5097,  1114,  3087,  1105,  1917,  1950,  1110,  7630,   119, 19984,
          1413,   131,  1191,  1128,  1178,  1328,  1141,  1520,   117,  1103,
          4995, 13929,  1448,   119,   119,   119,  1110,  1930,   170,  1618,
          3026,   117,  1649,   117,  1191,  1128,  1176,  7129,  1176,   146,
          1176,  7129,   117,  1128,  1281,   112,   189,  1301,  2488,  1113,
          1142,  1141,  1719,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0,

tensor([[0.2984, 0.3308, 0.3708]], grad_fn=<SoftmaxBackward0>)

In [10]:
sequence_1 = "Not much I can say about this book... It's full of VVVery difficult number puzzles... at least for me. Okay, so I don't get 'A's for my grades at school, but I think that even you might find it a little difficult. The way I see it, you will need lots of paper, patience and time, say 30 minutes or so to solve even the easiest of these.Why? Because basically, you are asked how many combinations can be formed from such and such. Example: How many ways get 31241 by adding prime numbers only? This is not really what the problems are like in the book, they are far more complicated than that.For me, it's just too much. I can't even do 1 single problem. I can't even look up the answers because uses some sort of a legend that I can't understand. But if you have an above average intelligence or are mathematically gifted, you will probably find some obscure short cut to the problem a la Gauss. For the rest, forget it.Word of warning: these are purely number problems. I can't tell if logic will help or even plays a part in this book, but the problems are NUMBER problems."
seq1_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq1_tokens['input_ids'].flatten())}")
F.softmax(model(**seq1_tokens).logits, dim=1)

number of tokens in seq1 is 270


tensor([[0.3430, 0.3177, 0.3393]], grad_fn=<SoftmaxBackward0>)

In [11]:
sequence_2 = "I bought this book because I read some glowing praise on an online library site. Unfortunately, I was deeply disappointed by page three. I always buy books in the hope and expectation of having an enjoyable read, not to criticise. However, this book is in urgent need of good editing -- though quite possibly editing alone wouldn't save it. Examples: a bed squeaks slightly and sharply in the same sentence; a nightgown hangs freely over her girlish figure and olive colored complexion; coffee aromas huddle; rumbling clouds huddle (as well as the coffee?); she prepared to sip her coffee beneath the wrath of God... cuddled within the arms of a strong breeze; (the wrath of God is a breeze?); the Columbian (stet) coffee aroma danced beneath her sculpted tan nose; the coffee bean fragrance tangoed within her body; she placed her thick pink lips against the warm cup;... and so on, all by page three. It is quite possible that the storyline is deeply moving. I'll never know because I can't bring myself to continue. Sorry."
seq2_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq2_tokens['input_ids'].flatten())}")
F.softmax(model(**seq2_tokens).logits, dim=1)

number of tokens in seq2 is 244


tensor([[0.3258, 0.3109, 0.3633]], grad_fn=<SoftmaxBackward0>)

In [12]:
sequence_3 = "This book appears to be a print on demand style book which manifests itself, in this case, as a poor quality hardcover. Some of the text is laughably bad, the few images near the end look like they are snapshots from a black and white TV screen, but most bothersome is the muddy look of the text. The price seems a bit steep for such a poor quality print. That said, the actual content is very comprehensive. One of the authors (Gray) is co-creator of the now commonplace LBG (Linde Buzo Gray) VQ algorithm. Fortunately, the quality of the content shines through the terrible printing."
seq3_tokens = tokenizer(sequence_3, return_tensors="pt")
print(f"number of tokens in seq3 is {len(seq3_tokens['input_ids'].flatten())}")
F.softmax(model(**seq3_tokens).logits, dim=1)

number of tokens in seq3 is 138


tensor([[0.3349, 0.3320, 0.3331]], grad_fn=<SoftmaxBackward0>)

In [13]:
df['review/text'] = df['review/text'].astype(str)
df = df[~df['review/text'].isnull()]
len(df)

#df['review/text'] = df['review/text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#def remove_punctuation(text):
    #return re.sub(r'[^\w\s]', '', text)

#df['review/text'] = df['review/text'].apply(remove_punctuation)

#stop = stopwords.words('english')
#df['review/text'] = df['review/text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

df.head()

,Title,review/score,review/text,sentiment,label
1361968,The Secret Garden (Worlds Classics),5.0,I was excited to find this book accessible and...,positive,0
612376,Christ and culture,5.0,"H. Richard Niebuhr writes as a Christian, but ...",positive,0
2415963,Raging Bull,5.0,My title may be entrenched in the idea that wh...,positive,0
2093063,Wuthering Heights,5.0,I am trying to read the classics now. I rememb...,positive,0
2958726,"Lord of the Flies, a Novel",5.0,"I read Lord of the Flies in my english class, ...",positive,0


In [14]:
ds_raw = Dataset.from_pandas(df[['label','review/text']])
ds_raw[0]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


{'label': 0,
 'review/text': 'I was excited to find this book accessible and free for my new kindle device. It really was fabulous to read it again as an adult. The Secret Garden is still a booik that makes you imagine, wonder, and be truly grateful for everything you have, and see "magic" in our own everydays.',
 '__index_level_0__': 1361968}

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["review/text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/30 [00:00<?, ?ba/s]

In [16]:
ds[0]

{'label': 0,
 'review/text': 'I was excited to find this book accessible and free for my new kindle device. It really was fabulous to read it again as an adult. The Secret Garden is still a booik that makes you imagine, wonder, and be truly grateful for everything you have, and see "magic" in our own everydays.',
 '__index_level_0__': 1361968,
 'input_ids': [101,
  146,
  1108,
  7215,
  1106,
  1525,
  1142,
  1520,
  7385,
  1105,
  1714,
  1111,
  1139,
  1207,
  1912,
  1513,
  4442,
  119,
  1135,
  1541,
  1108,
  175,
  20356,
  1106,
  2373,
  1122,
  1254,
  1112,
  1126,
  4457,
  119,
  1109,
  7013,
  5217,
  1110,
  1253,
  170,
  171,
  5658,
  4847,
  1115,
  2228,
  1128,
  5403,
  117,
  4608,
  117,
  1105,
  1129,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [17]:
ds = ds.shuffle(seed=42)
ds[0]

{'label': 0,
 'review/text': "I was first introdced to Solzhenitsyn's works when I was a freshman in high school, far too many years ago in a little town. The book was the Volume 1 of The Gulag Archipelago. It was really an eye-opener for me in so many ways, given that it was the first &quot;really serious&quot; book that I'd read.I believe that Solzhenitsyn is the best writer of the 20th century, or at least he's the top writer I've read so far (and I've read a lot of books). Maybe that's influenced by my early exposure, but I don't think so; I find his works just as compelling now as I did then.The First Circle is one of his most &quot;accessible&quot; works (that is, you can just jump in and start reading) and probably one of his best. A very compelling story; his portraits of the various vile creatures of the Soviet government have been shown to be quite accurate, and the way the various plots intertwine and are resolved is wonderful.The First Circle gives great insight into a cult

In [18]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [19]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 25500
len(ds_eval) = 4500


In [20]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [21]:
import os
os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
#os.environ["WANDB_DISABLED"] = "true"

In [22]:
import wandb
wandb.init(project='AmazonBookReviewDeepLearningModel',entity='iruz')
wandb.config ={
    'eval_steps': 500,
    'learning_rate':1e-5,
    'train_batch_size':32,
    'eval_batch_size':32
}

wandb: Currently logged in as: iruz. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231203_045746-s4woln9k
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ancient-snow-14
wandb: ⭐️ View project at https://wandb.ai/iruz/AmazonBookReviewDeepLearningModel
wandb: 🚀 View run at https://wandb.ai/iruz/AmazonBookReviewDeepLearningModel/runs/s4woln9k


In [23]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to='wandb',
                                  output_dir="/kaggle/working",
                                  evaluation_strategy="steps",
                                  eval_steps=500,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

In [24]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else: 
    device = "cpu"

Using GPU


In [25]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [26]:
torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy
500,0.529600,0.496540,0.834222
1000,0.439100,0.472224,0.840667
1500,0.384700,0.457989,0.841778
2000,0.323800,0.538176,0.835333
2500,0.291400,0.538634,0.833556
3000,0.242100,0.553914,0.824444
3500,0.202600,0.615603,0.830889
4000,0.187600,0.648464,0.823333
4500,0.145800,0.690170,0.822444
5000,0.139200,0.758867,0.821333


{'eval_loss': 0.8917784094810486,
 'eval_accuracy': 0.8188888888888889,
 'eval_runtime': 8.0037,
 'eval_samples_per_second': 562.242,
 'eval_steps_per_second': 17.617,
 'epoch': 10.0}

In [27]:
wandb.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  eval/accuracy ▆██▇▆▄▆▄▄▃▄▄▂▃▁▃
wandb:                      eval/loss ▂▁▁▂▂▃▃▄▅▆▆▇▇███
wandb:                   eval/runtime ▃▂▇▇▁▁▄▆▄▅▆▆▇▄█▁
wandb:        eval/samples_per_second ▆▇▂▂██▅▃▅▄▃▃▂▅▁█
wandb:          eval/steps_per_second ▆▇▂▂██▅▃▅▄▃▃▂▅▁█
wandb:                    train/epoch ▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
wandb:              train/global_step ▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
wandb:            train/learning_rate █▇▇▆▆▅▅▄▄▃▃▃▂▁▁
wandb:                     train/loss █▇▆▅▄▄▃▃▂▂▂▁▁▁▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                  eval/accuracy 0.81889
wandb:                      eval/loss 0.89178
wandb:                   eval/runtime 8.